<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Main-Cycle" data-toc-modified-id="Main-Cycle-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Main Cycle</a></span><ul class="toc-item"><li><span><a href="#Generate-Mapping" data-toc-modified-id="Generate-Mapping-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Generate Mapping</a></span></li><li><span><a href="#Process-Documents" data-toc-modified-id="Process-Documents-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Process Documents</a></span><ul class="toc-item"><li><span><a href="#My-matheuristic-Papers" data-toc-modified-id="My-matheuristic-Papers-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>My matheuristic Papers</a></span></li><li><span><a href="#Read-Abstract" data-toc-modified-id="Read-Abstract-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Read Abstract</a></span></li><li><span><a href="#Read-Entire-File" data-toc-modified-id="Read-Entire-File-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>Read Entire File</a></span></li><li><span><a href="#From-Arxiv.org" data-toc-modified-id="From-Arxiv.org-1.2.4"><span class="toc-item-num">1.2.4&nbsp;&nbsp;</span>From Arxiv.org</a></span></li></ul></li></ul></li><li><span><a href="#Doc2vec-Creation" data-toc-modified-id="Doc2vec-Creation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Doc2vec Creation</a></span></li><li><span><a href="#Using-the-doc2vec-model" data-toc-modified-id="Using-the-doc2vec-model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Using the doc2vec model</a></span></li><li><span><a href="#Populate-Lists" data-toc-modified-id="Populate-Lists-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Populate Lists</a></span></li><li><span><a href="#Testing" data-toc-modified-id="Testing-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Testing</a></span></li></ul></div>

In [46]:
import io
import os
#pdf2txt !!!!
import PyPDF2 as pyPdf

from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.converter import PDFPageAggregator
from pdfminer.pdfpage import PDFPage
from pdfminer.layout import LTTextBoxHorizontal

import re
import numpy as np
import pandas as pd
from collections import namedtuple
import matplotlib.pyplot as plt

import shutil
from os import path, listdir
from nltk import word_tokenize
from nltk.util import ngrams
import itertools
import pickle
import glob

# remove plurals
import inflect
p = inflect.engine()
#p.classical() # switch on classical plurals. E.g.: focus --> foci
p.defnoun( "class", "class" )
p.defnoun( "basis", "basis" )
p.defnoun("numerous", "numerous")
p.defnoun("focus", "focus")
p.defnoun("cross", "cross")
p.defnoun("success", "success")
p.defnoun("address", "address")
p.defnoun("pernicious", "pernicious")
p.defnoun("various", "various")
p.defnoun("continuous", "continuous")
p.defnoun("obvious", "obvious")
p.defnoun("process", "process")
p.defnoun("analysis", "analysis")
p.defnoun("enormous", "enormous")
p.defnoun("previous", "previous")
p.defnoun("homogeneous", "homogeneous")
p.defnoun("heterogeneous", "heterogeneous")
p.defnoun("tardiness", "tardiness")
p.defnoun("uniqueness", "uniqueness")
p.defnoun("fitness", "fitness")
p.defnoun("campus", "campus")
p.defnoun("relatedness", "relatedness")
p.defnoun("randomness", "randomness")
p.defnoun("express", "express")

p.defnoun("slackness", "slackness")
p.defnoun("simultaneous", "simultaneous")
p.defnoun("usefulness", "usefulness")
p.defnoun("synchronous", "synchronous")
p.defnoun("asynchronous", "asynchronous")
p.defnoun("hypothesis", "hypothesis")
p.defnoun("involves", "involves")
p.defnoun("witness", "witness")
p.defnoun("useless", "useless")
p.defnoun("correctness", "correctness")
p.defnoun("effectiveness", "effectiveness")
p.defnoun("exactness", "exactness")


#from utils import Config
import utils
import importlib
importlib.reload(utils)

<module 'utils' from '/home/mcaserta/research/matheur/utils.py'>

In [25]:
from nltk.corpus import stopwords
stop_words = stopwords.words("english")
stop_words += ['abstract', 'a', 'about', 'above', 'across', 'after', 'afterwards']
stop_words += ['again', 'against', 'all', 'almost', 'alone', 'along']
stop_words += ['already', 'also', 'although', 'always', 'am', 'among']
stop_words += ['amongst', 'amoungst', 'amount', 'an', 'and', 'another']
stop_words += ['any', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere']
stop_words += ['are', 'around', 'as', 'at', 'back', 'be', 'became']
stop_words += ['because', 'become', 'becomes', 'becoming', 'been']
stop_words += ['before', 'beforehand', 'behind', 'being', 'below']
stop_words += ['beside', 'besides', 'between', 'beyond', 'bill', 'both']
stop_words += ['bottom', 'but', 'by', 'call', 'can', 'cannot', 'cant']
stop_words += ['co', 'computer', 'con', 'could', 'couldnt', 'cry', 'de']
stop_words += ['describe', 'detail', 'did', 'do', 'done', 'down', 'due']
stop_words += ['during', 'each', 'eg', 'eight', 'either', 'eleven', 'else']
stop_words += ['elsewhere', 'empty', 'enough', 'etc', 'even', 'ever']
stop_words += ['every', 'everyone', 'everything', 'everywhere', 'except']
stop_words += ['few', 'fifteen', 'fifty', 'fill', 'find', 'fire', 'first']
stop_words += ['five', 'for', 'former', 'formerly', 'forty', 'found']
stop_words += ['four', 'from', 'front', 'full', 'further', 'get', 'give']
stop_words += ['go', 'had', 'has', 'hasnt', 'have', 'he', 'hence', 'her']
stop_words += ['here', 'hereafter', 'hereby', 'herein', 'hereupon', 'hers']
stop_words += ['herself', 'him', 'himself', 'his', 'how', 'however']
stop_words += ['hundred', 'i', 'ie', 'if', 'in', 'inc', 'indeed']
stop_words += ['interest', 'into', 'is', 'it', 'its', 'itself', 'keep']
stop_words += ['last', 'latter', 'latterly', 'least', 'less', 'ltd', 'made']
stop_words += ['many', 'may', 'me', 'meanwhile', 'might', 'mill', 'mine']
stop_words += ['more', 'moreover', 'most', 'mostly', 'move', 'much']
stop_words += ['must', 'my', 'myself', 'name', 'namely', 'neither', 'never']
stop_words += ['nevertheless', 'next', 'nine', 'no', 'nobody', 'none']
stop_words += ['noone', 'nor', 'not', 'nothing', 'now', 'nowhere', 'of']
stop_words += ['off', 'often', 'on','once', 'one', 'only', 'onto', 'or']
stop_words += ['other', 'others', 'otherwise', 'our', 'ours', 'ourselves']
stop_words += ['out', 'over', 'own', 'part', 'per', 'perhaps', 'please']
stop_words += ['put', 'rather', 're', 's', 'same', 'see', 'seem', 'seemed']
stop_words += ['seeming', 'seems', 'serious', 'several', 'she', 'should']
stop_words += ['show', 'side', 'since', 'sincere', 'six', 'sixty', 'so']
stop_words += ['some', 'somehow', 'someone', 'something', 'sometime']
stop_words += ['sometimes', 'somewhere', 'still', 'such', 'system', 'take']
stop_words += ['ten', 'than', 'that', 'the', 'their', 'them', 'themselves']
stop_words += ['then', 'thence', 'there', 'thereafter', 'thereby']
stop_words += ['therefore', 'therein', 'thereupon', 'these', 'they']
stop_words += ['thick', 'thin', 'third', 'this', 'those', 'though', 'three']
stop_words += ['three', 'through', 'throughout', 'thru', 'thus', 'to']
stop_words += ['together', 'too', 'top', 'toward', 'towards', 'twelve']
stop_words += ['twenty', 'two', 'un', 'under', 'unless', 'until', 'up', 'upon']
stop_words += ['us', 'very', 'via', 'was', 'we', 'well', 'were', 'what']
stop_words += ['whatever', 'when', 'whence', 'whenever', 'where']
stop_words += ['whereafter', 'whereas', 'whereby', 'wherein', 'whereupon']
stop_words += ['wherever', 'whether', 'which', 'while', 'whither', 'who']
stop_words += ['whoever', 'whole', 'whom', 'whose', 'why', 'will', 'with']
stop_words += ['within', 'without', 'would', 'yet', 'you', 'your']
stop_words += ['yours', 'yourself', 'yourselves']

**NOTE**: All words in the dictionary are transformed in lower case!

In [26]:
fullpath = path.join(utils.Config.prefixDict,utils.Config.vocab_folder)
name_vocab = fullpath + "embed500.vocab"
#name_vocab = fullpath + "embed.vocab"

with open(name_vocab) as f:
    vocab_list = map(str.strip,f.readlines())
vocab_dict = {w.lower():k for k,w in enumerate(vocab_list)} # transform all words in lower case



In [4]:
ww = "nature-designed"
print("Test = '{0}' in dict ? {1}".format(ww, ww in vocab_dict))
print(vocab_dict["aa"])

Test = 'nature-designed' in dict ? True
155935


Below, we eliminate the "-" from hyphenation. I thought it was going to be more complex, to deal with cases like "state-of-the-art." However, I observed that, when the hypenation is joining multiple words, these are read and N-grams (a single word, e.g., "worst-case.") Thus, nothing needs to be done in these cases. 

What we need to fix is the hyphenation at the end of the line. The pdf file might separate words to deal with the end of line. In these cases, we just need to join two consecutive words. Thus, the while cycle below is actually never used, since precisely two "words" are neeeded to recompose the original word.

Here we have *a little issue*. Since the chunk of text we obtain from the pdf does not extend over multiple pages, it could happen that a word is split and separated over two lines belonging to different pages. For example, imagine that the first page ends with "mor-" and teh second page starts with "ning". In this case, since the chunk finishes with the last word of the page, we are not able to "look at the next word" and thus join "morn-" with whatever follows. Of course, this could be fixed, but I do not think it is so important. In the current version, we simply skip the last word with the hyphen, if it is the last word of a chunk.

In [27]:
def create_dict(fileToOpen, lower=True):
    with open(fileToOpen, "r") as ff:
        my_list = map(str.strip, ff.readlines())
    if lower:
        my_dict = {w.lower():k for k,w in enumerate(my_list)}
    else:
        my_dict = {w:k for k,w in enumerate(my_list)}

    return my_dict

def create_acrons_dict(fileToOpen):
    my_dict = dict()
    with open(fileToOpen, "r") as ff:
        for row in ff:
            #row = ff.readline()
            #print("row=", row)
            row = word_tokenize(row)
            my_dict[row[0]] = row[1]
    
    return my_dict

This chunk is used to load the list of N-grams built while preprocessing all the documents in the first run. The idea is to build such list of bi-grams and use them to transform disjoint words into bi-grams. For example, many people might say *worst case*, while others might use *worst-case*. We transform all the ocurrences into *worst-case*. In addition, it makes sense to transform some key sentences into n-grams. Some examples are:
* mathematical programming --> mathematical_programming
* data mining
* operations research
* management science
* corridor method
* tabu search
* genetic algorithm(s)

The idea is to avoid counting management and science separately. Rather, we want it to be treated as a unique word.


In [10]:
ngrams_dict  = create_dict(utils.Config.ngramsFile, True)
acrons_dict  = create_acrons_dict(utils.Config.acronsFile) # do not transform to lower case 
special_dict = create_acrons_dict(utils.Config.specialFile) # do not transform to lower case 

chunk = "mathematical primal and dual for lagrangian and Bound, and Lagrangian two branch-and-cut math-heuristics based on the model"
chunk = re.sub('[“”,.’´`{}()&%\[\]]','', chunk)
print("Chunk here =", chunk)
chunk = convert_special_words(chunk, special_dict)
print("Chunk after =", chunk)
chunk = change_acronyms(chunk, acrons_dict)
words = [w.lower() for w in word_tokenize(chunk) if w.lower() not in stop_words]
newWords = [w.split("-") for w in words]
newWords = list(itertools.chain.from_iterable(newWords))
chunk = ' '.join(newWords) # re-join words and substitute
print("Now chunk is ;; ", chunk)    

print("WORDS = ", words)
print("NEWWORDS = ", newWords)
words = include_ngrams(newWords, ngrams_dict)
#print("CHUNK = ", chunk)
print("WORDS = ", words)

Chunk here = mathematical primal and dual for lagrangian and Bound and Lagrangian two branch-and-cut math-heuristics based on the model
Chunk after = mathematical primal and dual for lagrangean and Bound and lagrangean two branch-cut matheuristic based on the model
Now chunk is ;;  mathematical primal dual lagrangean bound lagrangean branch cut matheuristic based model
WORDS =  ['mathematical', 'primal', 'dual', 'lagrangean', 'bound', 'lagrangean', 'branch-cut', 'matheuristic', 'based', 'model']
NEWWORDS =  ['mathematical', 'primal', 'dual', 'lagrangean', 'bound', 'lagrangean', 'branch', 'cut', 'matheuristic', 'based', 'model']
WORDS =  ['mathematical', 'primal-dual', 'lagrangean', 'bound', 'lagrangean', 'branch-cut', 'matheuristic', 'based', 'model']


In [28]:
def include_ngrams(newWords, ngrams_dict):
    newWords = [w.split("-") for w in newWords]
    newWords = list(itertools.chain.from_iterable(newWords))
    chunk = ' '.join(newWords) # re-join words and substitute
    # look for n-grams
    for ngram in ngrams_dict:
        #print("Searching for ", ngram)
        #ngram = re.sub('-', ' ', ngram)
        #print("... in chunk ", chunk)
        auxChunk = re.sub('-', ' ', chunk) # to address cases as iterated local-search
        block = re.compile(ngram, re.IGNORECASE).finditer(auxChunk)

        init   = []
        ending = []
        for m in block:
            init.append(m.start())
            ending.append(m.end())
        #print("init and end ", init, ending)
        if len(init) > 0:
            #print("Changing ", ngram)
            i = 0
            sentence = chunk[:init[0]]
            for i in range(len(init)):
                sentence += re.sub(' ','-',chunk[init[i]:ending[i]])
                if i < len(init) -1 :
                    sentence += chunk[ending[i]:init[i+1]]
                else:
                    sentence += chunk[ending[i]:]
            chunk = sentence
            newWords = [w for w in word_tokenize(sentence)]
            #print("Now we have ", newWords)
            #print("** N-gram found : ", ngram)
            #input("aka")
    
    return newWords
          
def change_acronyms(chunk, acrons_dict):
    for acron in acrons_dict:
        re.sub('-',' ',acrons_dict[acron])
        chunk = re.sub(' ' + acron + ' ', ' ' + acrons_dict[acron] + ' ', chunk)
    return chunk

def convert_special_words(chunk, special_dict):
    for special in special_dict:
        chunk = re.sub(special, special_dict[special], chunk, flags=re.I) #ignore case
    return chunk
        

# list of fixes due to OCR errors
OCRFixes = {'ﬁ'  : 'fi',
            'ﬀ'  : 'ff',
            'ﬂ'  : 'fl',
            'ﬃ' : 'ffi'
}
replacementDictRegex = re.compile("(%s)" % "|".join(map(re.escape, OCRFixes.keys())))


def fix_OCR_errors(chunk):
    #chunk = re.sub('ﬁ','fi', chunk) 
    chunk = replacementDictRegex.sub(lambda mo:OCRFixes.get(mo.group(1),mo.group(1)), chunk)
    return chunk


def process_chunk(chunk, ngrams_dict, acrons_dict, special_dict):

    chunk = fix_OCR_errors(chunk)
    chunk = re.sub('[“”,.’´`{}()&%\[\]]','', chunk)
    chunk = re.sub('/', ' ', chunk)
    chunk = convert_special_words(chunk, special_dict)
    chunk = change_acronyms(chunk, acrons_dict)

    words = [w.lower() for w in word_tokenize(chunk) if w.lower() not in stop_words]
    newWords = []
    i = 0
    while i < len(words): # skip last element (we could lose a few words here)
        w = words[i]
        if len(w) <= 1:
            i += 1
            continue
        if i < len(words)-1:
            concat = ""
            if w[-1] == "-":
                concat += w[:-1]
                i += 1
                concat += words[i] # go to next word and join
                #print("** Concatenated word = ", concat)
                w = concat
        
        newWords.append(w)
        i += 1
        
    # convert nouns to singular
    newWords = [p.singular_noun(w) if p.singular_noun(w) else w for w in newWords]

    # eliminate all bi-grams (we recreated them according to our list)
    if len(newWords) > 0:
        newWords = include_ngrams(newWords, ngrams_dict)
        
    newWords = add_chunk_to_corpus(newWords)
    return newWords


def add_chunk_to_corpus(words):
    corrected = []  
    toAdd = False
    for w in words:    
        if (len(re.findall("-",w)) > 0 or (w.isalpha()) and len(w) > 2 and w in vocab_dict): # eliminate short words
#            if w not in vocab_dict:
#                print("?? {0} not in dict".format(w))
#                toAdd = True
            # if it is a composed word, e.g., "worst-case" or is alphabetical (note that "worst-case" is NOT alpha)
#            if w not in vocab_dict:
#                print("Word : ",w, "not in dictionary")

            corrected.append(w)
#    if toAdd:
#        input("check DICT")
    return corrected

In [7]:
def process_doc_info(doc):
    info = doc.info[0]
    #print(info)
    author = title = kw = ''
    if 'Author' in info.keys():
        #author = info["Author"].decode('utf-8')
        author = info["Author"].decode('iso-8859-1')
        #author = info["Author"]
    if 'Title' in info.keys():
        title  = info["Title"].decode('iso-8859-1')
    if 'Keywords' in info.keys():
        kw     = info["Keywords"].decode('iso-8859-1')
    return author, title, kw

# Main Cycle

Open a pdf file and process it. Check the following:
* proper elimination of hyphens at the end of the line
* proper dealing with n-grams
* generation of `myword.txt`, my dictionary of n-grams. This will be deactivated after a full first run

In reality, what we need to do is to re-read all the documents and introduce the n-grams (e.g., tabu_search, etc.)  We also need to deal with abbreviations, e.g.:
* GA --> genetic_algorithm
* TS --> tabu_search
* CM --> corridor_method

## Generate Mapping

In [41]:
year0   = "2001"
yearT   = "2019"
yearSet = list(range(int(year0), int(yearT)+1))

#df        = pd.DataFrame(columns=['id', 'year', 'name', 'nrWords', 'author', 'title', 'kw'])
df        = pd.DataFrame(columns=['id', 'year', 'name', 'nrWords'])
counter   = 0
for year in yearSet:
    print("** Year = ", year, "**", counter)
    listOfFiles = glob.glob(path.join(utils.Config.prefix,utils.Config.pdf_dir,str(year),"*.pdf"))

    for pdfFile in listOfFiles:

        #print("Opening FILE = ", pdfFile)
        filename = os.path.basename(pdfFile)
        name, file_extension = os.path.splitext(filename)
        filename = name + ".p"
        fullFileName = path.join(utils.Config.prefix, utils.Config.p_dir,str(year),filename)
        if os.path.isfile(fullFileName):
            aa = pickle.load(open(fullFileName, 'rb'))
            nrWords = len(aa)
            #document = open(pdfFile, 'rb')
            #parser   = PDFParser(document)
            #doc      = PDFDocument(parser)
            #author, title, kw = process_doc_info(doc)
            #df.loc[counter] = [counter, year, name, nrWords, author, title, kw]  
            df.loc[counter] = [counter, year, name, nrWords]   
            counter += 1
        else:
            print("File {0} does not exist.".format(fullFileName))

if not df.empty:
    df.to_csv(utils.Config.mappingFile)
print("Mapping saved as '{0}'. Total of {1} files.".format(utils.Config.mappingFile, counter))

** Year =  2001 ** 0
** Year =  2002 ** 2
** Year =  2003 ** 2
** Year =  2004 ** 4
** Year =  2005 ** 6
** Year =  2006 ** 8
** Year =  2007 ** 10
** Year =  2008 ** 12
** Year =  2009 ** 14
** Year =  2010 ** 19
** Year =  2011 ** 23
** Year =  2012 ** 27
** Year =  2013 ** 27
** Year =  2014 ** 36
** Year =  2015 ** 55
** Year =  2016 ** 71
** Year =  2017 ** 88
** Year =  2018 ** 112
** Year =  2019 ** 161
Mapping saved as 'mapping.csv'. Total of 194 files.


## Process Documents

### My matheuristic Papers

### Read Abstract

In [38]:
filename    = '/home/mcaserta/research/matheur/data/pAbstract/2017/hill-17-abs.p'
#ngrams_dict  = create_dict(utils.Config.ngramsFile, True)
#acrons_dict  = create_acrons_dict(utils.Config.acronsFile) # do not transform to lower case 
#special_dict = create_acrons_dict(utils.Config.specialFile) 

def manually_process_abstract():
    el="In the multi-depot ring star problem (MDRSP), a set of customers has to be connected to a set of given depots by ring stars. Such a ring star is a cycle graph, also called a ring, with some additional nodes assigned to its nodes by single star edges. Optional Steiner nodes can be used in the network as intermediate nodes on the rings. Depot dependent capacity limits apply to both, the number of customers in each ring star and the number of ring stars connected to a depot. The MDRSP asks for a network such that the sum of the edge costs is minimized. In this article, we present a matheuristic that iteratively refines a solution network in a locally exact fashion. In contrast to existing approaches, we define an equi-model matheuristic. That is a refinement method in which the subproblems are modeled as smaller instances of the global problem. Hence the optimization model that is used to explore the various structural multi-exchange neighborhoods in our algorithm is the MDRSP itself. A first class of neighborhoods considers local subnetworks for optimal improvements. Through an advanced modeling technique, we are able to refine arbitrary subnetworks of suitable size induced by simple node sets. A second class aims at globally restructuring the current network after the application of different contraction techniques. For both purposes, we develop an exact branch & cut algorithm for the MDRSP that efficiently solves the local optimization problems to optimality, if they are chosen reasonably in terms of size and complexity. The efficiency of the approach is shown by computational results improving known upper bounds for instance classes from the literature containing up to 1000 nodes. Ninety-one percent of the known best objective values are improved up to 13% in competitive computational time"
                   
    doc = []
    words = process_chunk(el, ngrams_dict, acrons_dict, special_dict)
    for w in words:
        doc.append(w)
    print("Saving file ", filename)
    print("*"*50,"\n")
    print("doc ", doc)
    pickle.dump(doc,open(filename, 'wb'))
    
manually_process_abstract()

Saving file  /home/mcaserta/research/matheur/data/pAbstract/2017/hill-17-abs.p
************************************************** 

doc  ['multi', 'depot', 'ring', 'star', 'problem', 'set', 'customer', 'connected', 'set', 'given', 'depot', 'ring', 'star', 'ring', 'star', 'cycle', 'graph', 'called', 'ring', 'additional', 'node', 'assigned', 'node', 'single', 'star', 'edge', 'optional', 'steiner', 'node', 'used', 'network', 'intermediate', 'node', 'ring', 'depot', 'dependent', 'capacity', 'limit', 'apply', 'number', 'customer', 'ring', 'star', 'number', 'ring', 'star', 'connected', 'depot', 'ask', 'network', 'sum', 'edge', 'cost', 'minimized', 'article', 'present', 'matheuristic', 'iteratively', 'refine', 'solution', 'network', 'locally', 'exact', 'fashion', 'contrast', 'existing', 'approach', 'define', 'equi', 'model', 'matheuristic', 'refinement', 'method', 'subproblem', 'modeled', 'smaller', 'instance', 'global', 'problem', 'optimization', 'model', 'used', 'explore', 'various', 'struc

In [47]:
if os.path.exists(os.path.dirname(utils.Config.mappingFile)):
    dfMap        = pd.read_csv(utils.Config.mappingFile)
    idCounter = dfMap["id"].max() + 1
else:
    idCounter = 0
    
#biblioTitle  = ["Bibliography", "References", "Acknowledgements"]
ngrams_dict  = create_dict(utils.Config.ngramsFile, True)
acrons_dict  = create_acrons_dict(utils.Config.acronsFile) # do not transform to lower case 
special_dict = create_acrons_dict(utils.Config.specialFile) 

year0   = "2001"
yearT   = "2019"
yearSet = list(range(int(year0), int(yearT)+1))

counter   = 0
for year in yearSet:
    listOfFiles = glob.glob(path.join(utils.Config.prefix,utils.Config.pdf_dir,str(year),"*.pdf"))
    #listOfFiles = ['/home/marco/gdrive/research/papers/2019/matheur/data/pdf/2008/puchinger-08.pdf']
    for pdfFile in listOfFiles:

        #print("Opening FILE = ", pdfFile)
        abstract_empty = False
        filename = os.path.basename(pdfFile)
        filename, file_extension = os.path.splitext(filename)
        filename += "-abs.p"
        fullFileName = path.join(utils.Config.prefix, utils.Config.pAbstract_dir,str(year),filename)
        if os.path.isfile(fullFileName):
            print("... Processed file {0} already exists in folder '{1}'. Skip this.".format(filename, path.join(utils.Config.pAbstract_dir, str(year))))
            continue
        #input("aka")
        else:
            print("+++ Processing file {0}. Destination folder ='{1}'.".format(filename, path.join(utils.Config.pAbstract_dir, str(year))))

        with open(pdfFile, 'rb') as ff: # I am not able to find a better way to get nr pages
            pdfAux = pyPdf.PdfFileReader(ff)
            totPages = pdfAux.getNumPages()
        
        document = open(pdfFile, 'rb')
        
        rsrcmgr     = PDFResourceManager() #Create resource manager
        laparams    = LAParams() # Set parameters for analysis.
        device      = PDFPageAggregator(rsrcmgr, laparams=laparams) # Create a PDF page aggregator object.
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        
        i        = 0
        doc      = []
        abstract   = False
        for page in PDFPage.get_pages(document):
            i += 1 # update page number in pdf
            #print("Processing page ", i)
            percentPages = i/int(totPages)
            
            interpreter.process_page(page)
            layout = device.get_result() # receive the LTPage object for the page.
            print(layout)
            for element in layout:
                if isinstance(element, LTTextBoxHorizontal):
                    el = element.get_text()
                    print(el)
                    
                    # next, attempt to skip bibliography
                    #if len(words) <= 5 and re.compile('|'.join(utils.Config.biblioTitle)).search(el):
                    if re.compile('|'.join(utils.Config.abstractTitle)).search(el) or abstract_empty==True:
                        if len(el) < 250:
                            abstract_empty = True
                            print("Reading next element to capture abstract...")
                        else:
                            print("{0:5d} :: Ref on page {1} = {2}".format(len(el), i, el)) 
                            abstract = True
                            words = process_chunk(el, ngrams_dict, acrons_dict, special_dict)
                            print(words)
                            for w in words:
                                doc.append(w)
                            input("aka")
                            break
                    
            if abstract:
                break
        if not os.path.exists(os.path.dirname(fullFileName)):
            os.makedirs(os.path.dirname(fullFileName))
        print("Saving file ", filename)
        print("*"*50,"\n")
        pickle.dump(doc,open(fullFileName, 'wb'))

        counter   += 1

print("Done with processing phase: Processed {0} files.".format(counter))

2019-03-20 15:52:35,660 : MainThread : INFO : xref found: pos=b'116'
2019-03-20 15:52:35,661 : MainThread : INFO : read_xref_from: start=116, token=/b'xref'
2019-03-20 15:52:35,663 : MainThread : INFO : xref objects: {1280: (None, 4424, 0), 1281: (None, 3367, 0), 1132: (None, 16, 0), 1133: (None, 4626, 0), 1134: (None, 4867, 0), 1135: (None, 4896, 0), 1136: (None, 4947, 0), 1137: (None, 4984, 0), 1138: (None, 5534, 0), 1139: (None, 5648, 0), 1140: (None, 5843, 0), 1141: (None, 5957, 0), 1142: (None, 6069, 0), 1143: (None, 6180, 0), 1144: (None, 6292, 0), 1145: (None, 6404, 0), 1146: (None, 6518, 0), 1147: (None, 6629, 0), 1148: (None, 6742, 0), 1149: (None, 6856, 0), 1150: (None, 6971, 0), 1151: (None, 7083, 0), 1152: (None, 7196, 0), 1153: (None, 7308, 0), 1154: (None, 7418, 0), 1155: (None, 7532, 0), 1156: (None, 7643, 0), 1157: (None, 7754, 0), 1158: (None, 7911, 0), 1159: (None, 8044, 0), 1160: (None, 8190, 0), 1161: (None, 8331, 0), 1162: (None, 8491, 0), 1163: (None, 8661, 0), 11

2019-03-20 15:52:35,681 : MainThread : INFO : trailer: {'Size': 1132}
2019-03-20 15:52:35,683 : MainThread : INFO : trailer: {'XRefStm': 4424, 'ID': [b'\xe2\xdayc\x86\x93\xd69U\xe7=I@\xe6$ ', b'\x80N\xaa\t^\xeb\x19A\xac,\x07\xe0\xa2\xe4\xaf\xc8'], 'Prev': 468614, 'Root': <PDFObjRef:1133>, 'Size': 1282, 'Info': <PDFObjRef:498>}
2019-03-20 15:52:35,686 : MainThread : INFO : Pages: Kids=[<PDFObjRef:495>, <PDFObjRef:496>]
2019-03-20 15:52:35,687 : MainThread : INFO : Pages: Kids=[<PDFObjRef:1137>, <PDFObjRef:1>, <PDFObjRef:25>, <PDFObjRef:56>, <PDFObjRef:82>, <PDFObjRef:103>, <PDFObjRef:111>, <PDFObjRef:127>, <PDFObjRef:150>, <PDFObjRef:164>]
2019-03-20 15:52:35,690 : MainThread : INFO : Page: {'StructParents': 1, 'Rotate': 0, 'MediaBox': [0, 0, 544.252, 742.677], 'CropBox': [0, 0, 544.252, 742.677], 'Type': /'Page', 'B': [<PDFObjRef:1164>, <PDFObjRef:1165>, <PDFObjRef:1203>, <PDFObjRef:1202>, <PDFObjRef:1201>, <PDFObjRef:1200>, <PDFObjRef:1199>, <PDFObjRef:1198>], 'Contents': [<PDFObjRef:

... Processed file taillard-01-abs.p already exists in folder 'data/pAbstract/2001'. Skip this.
... Processed file hansen-01-abs.p already exists in folder 'data/pAbstract/2001'. Skip this.
... Processed file dumitrescu-03-abs.p already exists in folder 'data/pAbstract/2003'. Skip this.
... Processed file fischetti-03-abs.p already exists in folder 'data/pAbstract/2003'. Skip this.
... Processed file klau-04-abs.p already exists in folder 'data/pAbstract/2004'. Skip this.
... Processed file fischetti-04-abs.p already exists in folder 'data/pAbstract/2004'. Skip this.
... Processed file danna-05-abs.p already exists in folder 'data/pAbstract/2005'. Skip this.
... Processed file puchinger-05-abs.p already exists in folder 'data/pAbstract/2005'. Skip this.
... Processed file sniedovich-06-abs.p already exists in folder 'data/pAbstract/2006'. Skip this.
... Processed file hansen-06-abs.p already exists in folder 'data/pAbstract/2006'. Skip this.
... Processed file rothberg-07-abs.p already

aka


2019-03-20 15:52:43,596 : MainThread : INFO : xref found: pos=b'251457'
2019-03-20 15:52:43,598 : MainThread : INFO : read_xref_from: start=251457, token=/b'xref'
2019-03-20 15:52:43,603 : MainThread : INFO : xref objects: {1: (None, 17, 0), 2: (None, 166, 0), 3: (None, 240, 0), 4: (None, 327, 0), 5: (None, 10340, 0), 6: (None, 10571, 0), 7: (None, 10727, 0), 8: (None, 26228, 0), 9: (None, 36066, 0), 10: (None, 42586, 0), 11: (None, 49035, 0), 12: (None, 54684, 0), 13: (None, 60714, 0), 14: (None, 65968, 0), 15: (None, 75737, 0), 16: (None, 75776, 0), 17: (None, 78471, 0), 18: (None, 78533, 0), 19: (None, 78586, 0), 20: (None, 79396, 0), 21: (None, 79486, 0), 22: (None, 79697, 0), 23: (None, 86693, 0), 24: (None, 87881, 0), 25: (None, 88097, 0), 26: (None, 90811, 0), 27: (None, 91992, 0), 28: (None, 92208, 0), 29: (None, 94925, 0), 30: (None, 96076, 0), 31: (None, 96280, 0), 32: (None, 97581, 0), 33: (None, 97703, 0), 34: (None, 98238, 0), 35: (None, 98279, 0), 36: (None, 105708, 0), 3

...

2019-03-20 15:52:44,004 : MainThread : INFO : Processing page: <PDFPage: Resources={'ExtGState': {'GS2': <PDFObjRef:17>, 'GS4': <PDFObjRef:18>}, 'XObject': {'X5': <PDFObjRef:721>, 'Im4': <PDFObjRef:700>, 'X6': <PDFObjRef:722>, 'X7': <PDFObjRef:723>, 'Im3': <PDFObjRef:697>, 'Im1': <PDFObjRef:693>, 'X2': <PDFObjRef:718>, 'X3': <PDFObjRef:719>, 'Im9': <PDFObjRef:711>, 'X10': <PDFObjRef:713>, 'Im2': <PDFObjRef:696>, 'X14': <PDFObjRef:717>, 'X9': <PDFObjRef:725>, 'X13': <PDFObjRef:716>, 'X11': <PDFObjRef:714>, 'X4': <PDFObjRef:720>, 'X1': <PDFObjRef:712>, 'Im8': <PDFObjRef:708>, 'FFT0': <PDFObjRef:743>, 'Im7': <PDFObjRef:705>, 'Im6': <PDFObjRef:704>, 'X12': <PDFObjRef:715>, 'Im5': <PDFObjRef:701>, 'X8': <PDFObjRef:724>}, 'Properties': {'MC1': <PDFObjRef:32>, 'MC2': <PDFObjRef:34>}, 'ProcSet': [/'PDF', /'Text', /'ImageC', /'ImageI', /'ImageB'], 'Font': {'F4': <PDFObjRef:29>, 'F2': <PDFObjRef:23>, 'F1': <PDFObjRef:19>, 'F3': <PDFObjRef:26>}, 'ColorSpace': {'Cs6': <PDFObjRef:15>}, 'Shading': {

2019-03-20 15:52:44,006 : MainThread : INFO : render_contents: resources={'ExtGState': {'GS2': <PDFObjRef:17>, 'GS4': <PDFObjRef:18>}, 'XObject': {'X5': <PDFObjRef:721>, 'Im4': <PDFObjRef:700>, 'X6': <PDFObjRef:722>, 'X7': <PDFObjRef:723>, 'Im3': <PDFObjRef:697>, 'Im1': <PDFObjRef:693>, 'X2': <PDFObjRef:718>, 'X3': <PDFObjRef:719>, 'Im9': <PDFObjRef:711>, 'X10': <PDFObjRef:713>, 'Im2': <PDFObjRef:696>, 'X14': <PDFObjRef:717>, 'X9': <PDFObjRef:725>, 'X13': <PDFObjRef:716>, 'X11': <PDFObjRef:714>, 'X4': <PDFObjRef:720>, 'X1': <PDFObjRef:712>, 'Im8': <PDFObjRef:708>, 'FFT0': <PDFObjRef:743>, 'Im7': <PDFObjRef:705>, 'Im6': <PDFObjRef:704>, 'X12': <PDFObjRef:715>, 'Im5': <PDFObjRef:701>, 'X8': <PDFObjRef:724>}, 'Properties': {'MC1': <PDFObjRef:32>, 'MC2': <PDFObjRef:34>}, 'ProcSet': [/'PDF', /'Text', /'ImageC', /'ImageI', /'ImageB'], 'Font': {'F4': <PDFObjRef:29>, 'F2': <PDFObjRef:23>, 'F1': <PDFObjRef:19>, 'F3': <PDFObjRef:26>}, 'ColorSpace': {'Cs6': <PDFObjRef:15>}, 'Shading': {'Sh516': <

2019-03-20 15:52:44,011 : MainThread : INFO : get_font: create: objid=29, spec={'BaseFont': /'QWEBYS+Springnew-Regular', 'Type': /'Font', 'LastChar': 255, 'FontDescriptor': <PDFObjRef:30>, 'Subtype': /'Type1', 'Encoding': /'WinAnsiEncoding', 'FirstChar': 0, 'Widths': [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 474, 215, 1970, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 0, 0, 1000, 1000, 1000, 1000, 1000, 0

2019-03-20 15:52:44,188 : MainThread : INFO : Processing xobj: <PDFStream(718): raw=639, {'BitsPerComponent': 8, 'Intent': /'Perceptual', 'Length': 637, 'Type': /'XObject', 'Filter': /'DCTDecode', 'Subtype': /'Image', 'Height': 2, 'Width': 2, 'ColorSpace': <PDFObjRef:15>}>
2019-03-20 15:52:44,194 : MainThread : INFO : Processing xobj: <PDFStream(719): raw=640, {'BitsPerComponent': 8, 'Intent': /'Perceptual', 'Length': 638, 'Type': /'XObject', 'Filter': /'DCTDecode', 'Subtype': /'Image', 'Height': 2, 'Width': 3, 'ColorSpace': <PDFObjRef:15>}>
2019-03-20 15:52:44,201 : MainThread : INFO : Processing xobj: <PDFStream(720): raw=641, {'BitsPerComponent': 8, 'Intent': /'Perceptual', 'Length': 639, 'Type': /'XObject', 'Filter': /'DCTDecode', 'Subtype': /'Image', 'Height': 2, 'Width': 3, 'ColorSpace': <PDFObjRef:15>}>
2019-03-20 15:52:44,207 : MainThread : INFO : Processing xobj: <PDFStream(721): raw=639, {'BitsPerComponent': 8, 'Intent': /'Perceptual', 'Length': 637, 'Type': /'XObject', 'Filt

<LTPage(1) 0.000,0.000,439.370,666.142 rotate=0>
4OR-Q J Oper Res (2015) 13:223–224
DOI 10.1007/s10288-014-0278-z

PHD THESIS

Uniﬁed matheuristic for solving rich vehicle
routing problems

Rahma Lahyani

Published online: 11 December 2014
© Springer-Verlag Berlin Heidelberg 2014

This is a summary of the author’s Ph.D. thesis jointly supervised by Frédéric Semet
and Mahdi Khemakhem and defended on 13 June 2014 at Ecole Centrale de Lille.
The thesis, written in English, is a joint work between Ecole Centrale de Lille and
Faculté des Sciences Economiques et de Gestion de Sfax, Tunisia. The document is
available from the author upon request at rahma.lahyani@gmail.com. The purpose of
this thesis is to develop a solution framework for multi-constrained vehicle routing
problems (VRPs).

In recent years, methodological progress and the development of computer tech-
nologies has led to an increasing academic attention to new variants of VRPs. This
research avenue is stimulated by the complex 

### Read Entire File

In [40]:
if os.path.exists(os.path.dirname(utils.Config.mappingFile)):
    dfMap        = pd.read_csv(utils.Config.mappingFile)
    idCounter = dfMap["id"].max() + 1
else:
    idCounter = 0
    
#biblioTitle  = ["Bibliography", "References", "Acknowledgements"]
ngrams_dict  = create_dict(utils.Config.ngramsFile, True)
acrons_dict  = create_acrons_dict(utils.Config.acronsFile) # do not transform to lower case 
special_dict = create_acrons_dict(utils.Config.specialFile) 

year0   = "2001"
yearT   = "2019"
yearSet = list(range(int(year0), int(yearT)+1))

counter   = 0
for year in yearSet:
    listOfFiles = glob.glob(path.join(utils.Config.prefix,utils.Config.pdf_dir,str(year),"*.pdf"))
    
    
    for pdfFile in listOfFiles:

        #print("Opening FILE = ", pdfFile)
        filename = os.path.basename(pdfFile)
        filename, file_extension = os.path.splitext(filename)
        filename += ".p"
        fullFileName = path.join(utils.Config.prefix, utils.Config.p_dir,str(year),filename)
        if os.path.isfile(fullFileName):
            print("... Processed file {0} already exists in folder '{1}'. Skip this.".format(filename, path.join(utils.Config.p_dir, str(year))))
            continue
        #input("aka")
        else:
            print("+++ Processing file {0}. Destination folder ='{1}'.".format(filename, path.join(utils.Config.p_dir, str(year))))

        with open(pdfFile, 'rb') as ff: # I am not able to find a better way to get nr pages
            pdfAux = pyPdf.PdfFileReader(ff)
            totPages = pdfAux.getNumPages()
        
        document = open(pdfFile, 'rb')
        
        rsrcmgr     = PDFResourceManager() #Create resource manager
        laparams    = LAParams() # Set parameters for analysis.
        device      = PDFPageAggregator(rsrcmgr, laparams=laparams) # Create a PDF page aggregator object.
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        
        i        = 0
        doc      = []
        biblio   = False
        for page in PDFPage.get_pages(document):
            i += 1 # update page number in pdf
            #print("Processing page ", i)
            percentPages = i/int(totPages)
            
            interpreter.process_page(page)
            layout = device.get_result() # receive the LTPage object for the page.
            for element in layout:
                if isinstance(element, LTTextBoxHorizontal):
                    el = element.get_text()
 
                    # next, attempt to skip bibliography
                    #if len(words) <= 5 and re.compile('|'.join(utils.Config.biblioTitle)).search(el):
                    if percentPages > 0.7 and re.compile('|'.join(utils.Config.biblioTitle)).search(el):
                        print("Ref on page ", i, " = ", el)                        
                        biblio = True
                        break
                        
                    print("\nBefore", el)
                    words = process_chunk(el, ngrams_dict, acrons_dict, special_dict)
                    print("After", words)

                    for w in words:
                        doc.append(w)
            if biblio:
                break
        if not os.path.exists(os.path.dirname(fullFileName)):
            os.makedirs(os.path.dirname(fullFileName))
        print("Saving file ", filename)
        print("*"*50,"\n")
        pickle.dump(doc,open(fullFileName, 'wb'))

        counter   += 1

print("Done with processing phase: Processed {0} files.".format(counter))

2019-03-20 15:38:32,356 : MainThread : INFO : xref found: pos=b'160'
2019-03-20 15:38:32,358 : MainThread : INFO : read_xref_from: start=160, token=/b'xref'
2019-03-20 15:38:32,359 : MainThread : INFO : xref objects: {768: (None, 143282, 0), 769: (None, 143403, 0), 770: (None, 143922, 0), 771: (None, 143963, 0), 772: (None, 164408, 0), 773: (None, 165004, 0), 774: (None, 165262, 0), 775: (None, 165630, 0), 776: (None, 170060, 0), 777: (None, 170446, 0), 778: (None, 170710, 0), 779: (None, 170825, 0), 780: (None, 171092, 0), 781: (None, 172642, 0), 782: (None, 172980, 0), 783: (None, 173922, 0), 784: (None, 174289, 0), 785: (None, 174757, 0), 786: (None, 181488, 0), 787: (None, 181847, 0), 788: (None, 182747, 0), 789: (None, 183108, 0), 790: (None, 183575, 0), 791: (None, 185223, 0), 792: (None, 185607, 0), 793: (None, 186086, 0), 794: (None, 186261, 0), 795: (None, 186568, 0), 796: (None, 189017, 0), 797: (None, 189378, 0), 798: (None, 189553, 0), 799: (None, 189766, 0), 800: (None, 18

2019-03-20 15:38:32,369 : MainThread : INFO : trailer: {'ID': [b'\xf5;\xfb(\x8eSe/bS\x0eZO\x111.', b'j\x9b\xe4q,[\x99\xf7c\xd8`\xf8\x9d\xd4\xe4\x05'], 'Root': <PDFObjRef:721>, 'Size': 720, 'Info': <PDFObjRef:717>}
2019-03-20 15:38:32,371 : MainThread : INFO : trailer: {'ID': [b'\xf5;\xfb(\x8eSe/bS\x0eZO\x111.', b'j\x9b\xe4q,[\x99\xf7c\xd8`\xf8\x9d\xd4\xe4\x05'], 'Prev': 1105957, 'Root': <PDFObjRef:721>, 'Size': 855, 'Info': <PDFObjRef:717>}
2019-03-20 15:38:32,374 : MainThread : INFO : Pages: Kids=[<PDFObjRef:715>, <PDFObjRef:716>]
2019-03-20 15:38:32,376 : MainThread : INFO : Pages: Kids=[<PDFObjRef:724>, <PDFObjRef:1>, <PDFObjRef:21>, <PDFObjRef:51>, <PDFObjRef:72>, <PDFObjRef:98>, <PDFObjRef:104>, <PDFObjRef:118>, <PDFObjRef:124>, <PDFObjRef:137>]
2019-03-20 15:38:32,378 : MainThread : INFO : Page: {'StructParents': 1, 'Rotate': 0, 'MediaBox': [0, 0, 595.245, 793.92], 'Parent': <PDFObjRef:715>, 'Type': /'Page', 'B': [<PDFObjRef:733>, <PDFObjRef:734>], 'Contents': [<PDFObjRef:725>, <

2019-03-20 15:38:32,417 : MainThread : INFO : get_font: create: objid=792, spec={'ToUnicode': <PDFObjRef:796>, 'BaseFont': /'KIBIHE+UniversLTStd', 'LastChar': 119, 'FirstChar': 46, 'FontDescriptor': <PDFObjRef:794>, 'Subtype': /'Type1', 'Encoding': <PDFObjRef:793>, 'Widths': [278, 278, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 278, 500, 500, 500, 500, 500, 500, 500, 500, 667, 722, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 667, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 556, 611, 500, 500, 556, 500, 611, 611, 278, 278, 500, 278, 944, 611, 611, 611, 500, 389, 500, 333, 611, 556, 833], 'Type': /'Font'}
2019-03-20 15:38:32,431 : MainThread : INFO : Processing xobj: <PDFStream(761): raw=48954, {'BitsPerComponent': 8, 'Type': /'XObject', 'Length': 48953, 'Filter': /'DCTDecode', 'Subtype': /'Image', 'Height': 273, 'Width': 249, 'Metadata': <PDFObjRef:762>, 'ColorSpace': /'DeviceRGB'}>
2019-03-20 15:38:32,435 : MainThread : INFO : Proce

... Processed file taillard-01.p already exists in folder 'data/p/2001'. Skip this.
... Processed file hansen-01.p already exists in folder 'data/p/2001'. Skip this.
... Processed file dumitrescu-03.p already exists in folder 'data/p/2003'. Skip this.
... Processed file fischetti-03.p already exists in folder 'data/p/2003'. Skip this.
... Processed file klau-04.p already exists in folder 'data/p/2004'. Skip this.
... Processed file fischetti-04.p already exists in folder 'data/p/2004'. Skip this.
... Processed file danna-05.p already exists in folder 'data/p/2005'. Skip this.
... Processed file puchinger-05.p already exists in folder 'data/p/2005'. Skip this.
... Processed file sniedovich-06.p already exists in folder 'data/p/2006'. Skip this.
... Processed file hansen-06.p already exists in folder 'data/p/2006'. Skip this.
... Processed file rothberg-07.p already exists in folder 'data/p/2007'. Skip this.
... Processed file boschetti-07.p already exists in folder 'data/p/2007'. Skip t

2019-03-20 15:38:33,518 : MainThread : INFO : Page: {'StructParents': 2, 'Rotate': 0, 'MediaBox': [0, 0, 595.245, 793.92], 'Parent': <PDFObjRef:715>, 'Type': /'Page', 'B': [<PDFObjRef:735>, <PDFObjRef:736>], 'Contents': <PDFObjRef:2>, 'Annots': [<PDFObjRef:8>, <PDFObjRef:9>, <PDFObjRef:10>, <PDFObjRef:11>, <PDFObjRef:12>, <PDFObjRef:13>, <PDFObjRef:14>, <PDFObjRef:15>, <PDFObjRef:16>, <PDFObjRef:17>, <PDFObjRef:18>, <PDFObjRef:19>, <PDFObjRef:20>], 'Resources': <PDFObjRef:3>, 'CropBox': [0, 0, 595.245, 793.92]}
2019-03-20 15:38:33,521 : MainThread : INFO : Processing page: <PDFPage: Resources={'ExtGState': {'GS1': <PDFObjRef:805>}, 'XObject': {'Im4': <PDFObjRef:4>}, 'Font': {'F7': <PDFObjRef:772>, 'F5': <PDFObjRef:782>, 'F2': <PDFObjRef:797>, 'F1': <PDFObjRef:800>}, 'Properties': {'MC17': <PDFObjRef:6>}, 'ProcSet': [/'PDF', /'Text', /'ImageC']}, MediaBox=[0, 0, 595.245, 793.92]>
2019-03-20 15:38:33,522 : MainThread : INFO : render_contents: resources={'ExtGState': {'GS1': <PDFObjRef:80

...


Before A.-K. Rothenbächer et al. / European Journal of Operational Research 255 (2016) 935–947 

After ['european', 'journal', 'operational', 'research']

Before 943 

After []

Before Table 2 
Comparison of lower bounds for different solving strategies. 

After ['table', 'comparison', 'lower-bound', 'different', 'solving', 'strategy']

Before Use bidirectional pricer 

After ['use', 'bidirectional', 'pricer']

Before True 

After ['true']

Before True 

After ['true']

Before False 

After ['false']

Before False 

After ['false']

Before Branch on sum of trains 

After ['branch', 'sum', 'train']

Before True 
True 
False 
False 

After ['true', 'true', 'false', 'false']

Before 1.57 (1.71) 
1.69 (1.91) 
0.13 (0.11) 
0.15 (0.38) 
True 

After ['true']

Before 4 9.34 (4 9.71) 
50.22 (51.13) 
42.20 (42.70) 
42.94 (43.29) 
False 

After ['false']

Before 1.51 (1.72) 
1.75 (2.10) 
0.22 (0.10) 
0.38 (0.29) 
True 

After ['true']

Before 4 9.54 (4 9.66) 
50.51 (50.77) 
42.33 (42.48) 
43.22

2019-03-20 15:39:08,257 : MainThread : INFO : Pages: Kids=[<PDFObjRef:153>, <PDFObjRef:164>, <PDFObjRef:185>]
2019-03-20 15:39:08,259 : MainThread : INFO : Page: {'StructParents': 11, 'Rotate': 0, 'MediaBox': [0, 0, 595.245, 793.92], 'Parent': <PDFObjRef:716>, 'Type': /'Page', 'B': [<PDFObjRef:752>, <PDFObjRef:753>], 'Contents': <PDFObjRef:154>, 'Resources': <PDFObjRef:155>, 'CropBox': [0, 0, 595.245, 793.92]}
2019-03-20 15:39:08,266 : MainThread : INFO : Processing page: <PDFPage: Resources={'ExtGState': {'GS1': <PDFObjRef:805>, 'GS2': <PDFObjRef:362>}, 'Font': {'F9': <PDFObjRef:347>, 'F10': <PDFObjRef:352>, 'F1': <PDFObjRef:800>, 'F13': <PDFObjRef:363>, 'F7': <PDFObjRef:772>, 'F4': <PDFObjRef:787>, 'F11': <PDFObjRef:356>, 'F2': <PDFObjRef:797>, 'F5': <PDFObjRef:782>}, 'Properties': {'MC79': <PDFObjRef:158>, 'MC83': <PDFObjRef:156>, 'MC78': <PDFObjRef:160>, 'MC84': <PDFObjRef:162>}, 'ProcSet': [/'PDF', /'Text']}, MediaBox=[0, 0, 595.245, 793.92]>
2019-03-20 15:39:08,267 : MainThread :


Before 944 

After []

Before A.-K. Rothenbächer et al. / European Journal of Operational Research 255 (2016) 935–947 

After ['european', 'journal', 'operational', 'research']

Before Table 4 
Average results for small instances, 6 instances per group. 

After ['table', 'average', 'result', 'small', 'instance', 'instance', 'group']

Before Instance group  | H |  n H 

After ['instance', 'group']

Before Time in 
seconds 

After ['time', 'second']

Before Gap 
(percent) 

After ['gap', 'percent']

Before  3 

After []

Before <

After []

Before #Gap 
percent 

After ['gap', 'percent']

Before Separation 
time 
(percent) 

After ['separation', 'time', 'percent']

Before Pricing 
time 
(percent) 

After ['pricing', 'time', 'percent']

Before Reopt. time 
(percent) 

After ['time', 'percent']

Before #Cuts  #Pricings  #Paths  #Nodes  #Only road 

After ['cut', 'pricing', 'path', 'node', 'road']

Before 100-split 
100-split 
100-split 
100-split 
100-split 
100-split 
500-split 
500-spli

2019-03-20 15:39:09,441 : MainThread : INFO : undefined: <PDFType1Font: basefont='KIBMAE+MTEX'>, 2
2019-03-20 15:39:09,449 : MainThread : INFO : undefined: <PDFType1Font: basefont='KIBMAE+MTEX'>, 2
2019-03-20 15:39:09,453 : MainThread : INFO : undefined: <PDFType1Font: basefont='KIBMAE+MTEX'>, 2
2019-03-20 15:39:09,467 : MainThread : INFO : undefined: <PDFType1Font: basefont='KIBMAE+MTEX'>, 2
2019-03-20 15:39:09,474 : MainThread : INFO : undefined: <PDFType1Font: basefont='KIBMAE+MTEX'>, 2
2019-03-20 15:39:09,484 : MainThread : INFO : undefined: <PDFType1Font: basefont='KIBMAE+MTEX'>, 2



Before A.-K. Rothenbächer et al. / European Journal of Operational Research 255 (2016) 935–947 

After ['european', 'journal', 'operational', 'research']

Before 945 

After []

Before Fig. 4.  Solution characteristics for the two large instances with different numbers of hubs. (For interpretation of the references to color in this ﬁgure, the reader is referred 
to the web version of this article.) 

After ['fig', 'solution', 'characteristic', 'large', 'instance', 'different', 'number', 'hub', 'interpretation', 'reference', 'color', 'figure', 'reader', 'referred', 'web', 'version', 'article']

Before number decreases strongly with increasing number of hubs, be- 
cause more hubs make the use of trains cheaper and faster. Cor- 

After ['number', 'decrease', 'strongly', 'increasing', 'number', 'hub', 'because', 'hub', 'make', 'use', 'train', 'cheaper', 'faster', 'cor']
Ref on page  11  =  Acknowledgments 

Saving file  rothenbaecher-18.p
**************************************************

### From Arxiv.org

Use this to process files from arxiv.org.

In [31]:
db = pickle.load(open(utils.Config.db_dir, "rb"))
df = pd.DataFrame(columns=['id', 'year', 'name', 'nrWords', 'title', 'author'])
counter = 1000
for item in db:
    #print(db[item])
    name = os.path.basename(db[item]["id"])
    ff = path.join(utils.Config.pArxiv_dir, name+".p")
    if os.path.isfile(ff):
        #print("Mapping file ", ff)
        doc = pickle.load(open(ff, 'rb'))
        nrWords = len(doc)
        df.loc[counter] = [counter, db[item]["published_parsed"][0], name, nrWords, db[item]["title"], db[item]["author"]]

        counter += 1
print(df)
df.to_csv(utils.Config.mappingArxiv) 
print("ArXiv map saved in file '{0}'.".format(utils.Config.mappingArxiv))

        id  year          name nrWords  \
1000  1000  2013   1305.0780v1    2737   
1001  1001  2009   0907.1298v2    1983   
1002  1002  2018  1807.10470v1     971   
1003  1003  2013   1303.6342v1    2623   
1004  1004  2018  1802.07342v1    4838   
1005  1005  2018  1807.01844v1    3732   
1006  1006  2017  1703.05206v2    4080   
1007  1007  2012   1204.0479v1    2832   
1008  1008  2012   1204.4051v1     394   
1009  1009  2016  1604.06901v1    3416   
1010  1010  2018  1808.10139v3    4239   
1011  1011  2015  1512.02369v1    3108   
1012  1012  2018  1803.09254v3    1510   
1013  1013  2013   1303.3154v3    2405   
1014  1014  2017  1707.05457v1    1986   
1015  1015  2013   1304.4861v1    2673   
1016  1016  2015  1511.06248v1    1895   
1017  1017  2013   1304.4051v1    2556   
1018  1018  2012   1211.0906v2    9941   
1019  1019  2012   1211.3371v1    4333   
1020  1020  2010   1003.2889v1    2217   
1021  1021  2015  1501.07814v1    3538   
1022  1022  2017  1707.06185v1    

**NOTE**: Change the following values if we want to use the second set of papers from arxiv.

In [28]:
utils.Config.mappingArxiv = 'mappingArxiv.csv'
utils.Config.arxiv_dir = os.path.join('data', 'pdf', 'arxiv')
utils.Config.pArxiv_dir = os.path.join('data', 'pArxiv')

In [22]:
if os.path.exists(os.path.dirname(utils.Config.mappingArxiv)):
    dfMap        = pd.read_csv(utils.Config.mappingArxiv)
    idCounter = dfMap["id"].max() + 1
else:
    idCounter = 0
    
ngrams_dict  = create_dict(utils.Config.ngramsFile, True)
acrons_dict  = create_acrons_dict(utils.Config.acronsFile) # do not transform to lower case 
special_dict = create_acrons_dict(utils.Config.specialFile) 

counter   = 0

listOfFiles = glob.glob(path.join(utils.Config.prefix,utils.Config.arxiv_dir,"*.pdf"))

for pdfFile in listOfFiles:
    
    try: 

        #print("Opening FILE = ", pdfFile)
        filename = os.path.basename(pdfFile)
        filename, file_extension = os.path.splitext(filename)
        filename += ".p"
        fullFileName = path.join(utils.Config.prefix, utils.Config.pArxiv_dir,filename)
        if os.path.isfile(fullFileName):      
        #if filename != "ren-14.p":
            print("... Processed file {0} already exists in folder '{1}'. Skip this.".format(filename, path.join(utils.Config.pArxiv_dir)))
            continue
        #input("aka")
        else:
            print("+++ Processing file {0}. Destination folder ='{1}'.".format(filename, path.join(utils.Config.pArxiv_dir)))

        with open(pdfFile, 'rb') as ff: # I am not able to find a better way to get nr pages
            pdfAux = pyPdf.PdfFileReader(ff)
            totPages = pdfAux.getNumPages()

        document = open(pdfFile, 'rb')

        rsrcmgr     = PDFResourceManager() #Create resource manager
        laparams    = LAParams() # Set parameters for analysis.
        device      = PDFPageAggregator(rsrcmgr, laparams=laparams) # Create a PDF page aggregator object.
        interpreter = PDFPageInterpreter(rsrcmgr, device)

        i        = 0
        doc      = []
        biblio   = False
        for page in PDFPage.get_pages(document):
            i += 1 # update page number in pdf
            #print("Processing page ", i)
            percentPages = i/int(totPages)

            interpreter.process_page(page)
            layout = device.get_result() # receive the LTPage object for the page.
            for element in layout:
                if isinstance(element, LTTextBoxHorizontal):
                    el = element.get_text()

                    # next, attempt to skip bibliography
                    #if len(words) <= 5 and re.compile('|'.join(utils.Config.biblioTitle)).search(el):
                    if percentPages > 0.5 and re.compile('|'.join(utils.Config.biblioTitle)).search(el):
                        print("Ref on page ", i, " = ", el)                        
                        biblio = True
                        break

                    #print("\nBefore", el)
                    words = process_chunk(el, ngrams_dict, acrons_dict, special_dict)
                    #print("After", words)

                    for w in words:
                        doc.append(w)
            if biblio:
                break
        if not os.path.exists(os.path.dirname(fullFileName)):
            os.makedirs(os.path.dirname(fullFileName))
        print("Saving file ", filename)
        print("*"*50,"\n")
        pickle.dump(doc,open(fullFileName, 'wb'))

        counter   += 1
    except:
        print("Error processing file", pdfFile,". Removed from disk.")
        os.remove(pdfFile)

print("Done with processing phase: Processed {0} files.".format(counter))

... Processed file 1408.3215v1.p already exists in folder 'data/pArxiv2'. Skip this.
... Processed file 1012.5815v2.p already exists in folder 'data/pArxiv2'. Skip this.
... Processed file 0903.0173v1.p already exists in folder 'data/pArxiv2'. Skip this.
... Processed file 1303.1271v5.p already exists in folder 'data/pArxiv2'. Skip this.
... Processed file 1611.08775v1.p already exists in folder 'data/pArxiv2'. Skip this.
... Processed file 1902.06228v1.p already exists in folder 'data/pArxiv2'. Skip this.
... Processed file 1303.6145v2.p already exists in folder 'data/pArxiv2'. Skip this.
... Processed file 1810.11134v1.p already exists in folder 'data/pArxiv2'. Skip this.
... Processed file 1401.3492v1.p already exists in folder 'data/pArxiv2'. Skip this.
... Processed file 1301.6972v1.p already exists in folder 'data/pArxiv2'. Skip this.
... Processed file 1701.08517v1.p already exists in folder 'data/pArxiv2'. Skip this.
... Processed file 1501.07814v1.p already exists in folder 'd

... Processed file 1602.01659v1.p already exists in folder 'data/pArxiv2'. Skip this.
... Processed file 1706.09106v2.p already exists in folder 'data/pArxiv2'. Skip this.
... Processed file 1901.00942v2.p already exists in folder 'data/pArxiv2'. Skip this.
... Processed file 1809.09284v1.p already exists in folder 'data/pArxiv2'. Skip this.
... Processed file 1701.05654v2.p already exists in folder 'data/pArxiv2'. Skip this.
... Processed file 1303.2215v1.p already exists in folder 'data/pArxiv2'. Skip this.
... Processed file 1210.7422v1.p already exists in folder 'data/pArxiv2'. Skip this.
... Processed file 1901.09831v2.p already exists in folder 'data/pArxiv2'. Skip this.
... Processed file 1803.09254v3.p already exists in folder 'data/pArxiv2'. Skip this.
... Processed file 1708.03703v1.p already exists in folder 'data/pArxiv2'. Skip this.
... Processed file 1409.6365v1.p already exists in folder 'data/pArxiv2'. Skip this.
... Processed file 1405.5643v1.p already exists in folder

In [45]:

df

,id,year,name,nrWords,author,title,kw
0,0,2001,taillard-01,1999,,popmusic3.dvi,
1,1,2001,hansen-01,2141,,Variable Neighborhood Decomposition Search,
2,2,2003,dumitrescu-03,5722,,,
3,3,2003,fischetti-03,3153,,,
4,4,2004,fischetti-04,2680,,,
5,5,2005,danna-05,3915,Springer-Verlag GmbH,Unbekannt,
6,6,2005,puchinger-05,2184,,,
7,7,2006,sniedovich-06,3784,,,
8,8,2006,hansen-06,1892,,doi:10.1016/j.cor.2005.02.033,
9,9,2007,rothberg-07,2513,,,


In [102]:
v = b'A matheuristic for aggregate production\x85distribution'
v.decode('iso-8859-1')

'A matheuristic for aggregate production\x85distribution'

Import mapping file to switch from name to id and viceversa.

In [10]:
def createMap(mappingFile):
    dfMap = pd.read_csv(mappingFile)
    #print(dfMap)
    file2tag = {f:t for f,t in zip(dfMap["name"],dfMap["id"])}
    tag2file = {t:f for t,f in zip(dfMap["id"],dfMap["name"])}
    print("Legths :: From file to TAG = {0}; from TAG to file = {1}".format(len(file2tag), len(tag2file)))
    #print("Name of file with id = {0} is {1}".format(1,tag2file[1]))
    #print("ID of file with name = {0} is {1}".format("puchinger-05", file2tag["puchinger-05"]))
    
    return file2tag, tag2file

In [22]:
def wordListToFreqDict(wordlist):
    wordfreq = [wordlist.count(p) for p in wordlist]
    return dict(zip(wordlist,wordfreq))

def sortFreqDict(freqdict):
    aux = [(freqdict[key], key) for key in freqdict]
    aux.sort()
    aux.reverse()
    return aux

# Doc2vec Creation

In [8]:
def create_corpus_core():
    
    file2tag, tag2file = createMap(utils.Config.mappingFile)
    analyzedDocument = namedtuple('AnalyzedDocument','words tags')
    year0 = "2001"
    yearT = "2019"
    yearSet = list(range(int(year0), int(yearT)+1))

    corpus = []
    counter  = 0
    txt = []
    for year in yearSet:
        #print("** Year = ", year, "**", counter)

        listOfFiles = glob.glob(path.join(utils.Config.prefix,utils.Config.p_dir,str(year),"*.p"))

        for pFile in listOfFiles:

            #print("Opening FILE = ", pFile)
            filename = os.path.basename(pFile)
            name, file_extension = os.path.splitext(filename)
            if os.path.isfile(pFile):
                ff = path.join(utils.Config.p_dir, str(year), filename)
                #print("Processing file '{0}'".format(ff))
                doc = pickle.load(open(ff, 'rb'))
                tags = [str(file2tag[name])]
                #print("... TAG = ", tags, " :: document length = ", len(doc), "words.")
                corpus.append(analyzedDocument(doc,tags))
                counter += 1
            else:
                print("** NOTE: File {0} not found!".format(filename))

    return corpus

def create_larger_corpus(corpus):
    
    # now add arXiv files
    totMine = len(corpus)
    counter = len(corpus)
    file2tagArxiv, tag2fileArxiv = createMap(utils.Config.mappingArxiv)
    listOfFiles = glob.glob(path.join(utils.Config.prefix,utils.Config.pArxiv_dir,"*.p"))

    for pFile in listOfFiles:

        #print("Opening FILE = ", pFile)
        filename = os.path.basename(pFile)
        name, file_extension = os.path.splitext(filename)
        if os.path.isfile(pFile):
            ff = path.join(utils.Config.pArxiv_dir, filename)
            #print("Processing file '{0}'".format(ff), "with name", name)
            doc = pickle.load(open(ff, 'rb'))
            #txt.append(doc)
            tags = [str(file2tagArxiv[name])]
            #print("... TAG = ", tags, " :: document length = ", len(doc), "words.")
            corpus.append(analyzedDocument(doc,tags))
            counter += 1
        else:
            print("** NOTE: File {0} not found!".format(ff))
        #input("aka")
    print("Loaded corpus with {0} documents [Mine ({1}) + Arxiv ({2})].".format(len(corpus), totMine, len(corpus)-totMine))
    
    return corpus

In [11]:
analyzedDocument = namedtuple('AnalyzedDocument','words tags')
corpus = create_corpus_core()

utils.Config.mappingArxiv = 'mappingArxiv.csv'
utils.Config.pArxiv_dir = os.path.join('data', 'pArxiv')
corpus = create_larger_corpus(corpus)

utils.Config.mappingArxiv = 'mappingArxiv2.csv'
utils.Config.pArxiv_dir = os.path.join('data', 'pArxiv2')
corpus = create_larger_corpus(corpus)

Legths :: From file to TAG = 193; from TAG to file = 193
Legths :: From file to TAG = 709; from TAG to file = 709
Loaded corpus with 902 documents [Mine (193) + Arxiv (709)].
Legths :: From file to TAG = 1613; from TAG to file = 1613
Loaded corpus with 2515 documents [Mine (902) + Arxiv (1613)].


In [12]:
from gensim.models import doc2vec
import logging
logging.basicConfig(
        format='%(asctime)s : %(threadName)s : %(levelname)s : %(message)s',
        level=logging.INFO
)


In [13]:
import multiprocessing
nCores = multiprocessing.cpu_count()
print("Using ", nCores, "cores.")

Using  32 cores.


In [14]:
#### creating doc2vec model for the current year
model = doc2vec.Doc2Vec(size=200, window=10, min_count=5, iter=250, workers=nCores, dm=1, negative=10)
model.build_vocab(corpus)
model.train(corpus, total_examples=model.corpus_count, epochs=model.iter)
model.init_sims(replace=True)

# save model on disk
fullname = path.join(utils.Config.models_dir,utils.Config.nameModel)
fullname = fullname + ".2"
print("Saving model ", fullname)
model.save(fullname)
model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

/home/mcaserta/anaconda3/lib/python3.5/site-packages/gensim/models/doc2vec.py:362: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
/home/mcaserta/anaconda3/lib/python3.5/site-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
2019-03-20 09:27:09,315 : MainThread : INFO : collecting all words and their counts
2019-03-20 09:27:09,317 : MainThread : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2019-03-20 09:27:11,466 : MainThread : INFO : collected 35617 word types and 1806 unique tags from a corpus of 2515 examples and 7721813 words
2019-03-20 09:27:11,467 : MainThread : INFO : Loading a fresh vocabulary
2019-03

2019-03-20 09:27:23,563 : MainThread : INFO : worker thread finished; awaiting finish of 18 more threads
2019-03-20 09:27:23,567 : MainThread : INFO : worker thread finished; awaiting finish of 17 more threads
2019-03-20 09:27:23,570 : MainThread : INFO : worker thread finished; awaiting finish of 16 more threads
2019-03-20 09:27:23,572 : MainThread : INFO : worker thread finished; awaiting finish of 15 more threads
2019-03-20 09:27:23,580 : MainThread : INFO : worker thread finished; awaiting finish of 14 more threads
2019-03-20 09:27:23,581 : MainThread : INFO : worker thread finished; awaiting finish of 13 more threads
2019-03-20 09:27:23,582 : MainThread : INFO : worker thread finished; awaiting finish of 12 more threads
2019-03-20 09:27:23,585 : MainThread : INFO : worker thread finished; awaiting finish of 11 more threads
2019-03-20 09:27:23,586 : MainThread : INFO : worker thread finished; awaiting finish of 10 more threads
2019-03-20 09:27:23,587 : MainThread : INFO : worker th

2019-03-20 09:27:35,364 : MainThread : INFO : worker thread finished; awaiting finish of 18 more threads
2019-03-20 09:27:35,379 : MainThread : INFO : worker thread finished; awaiting finish of 17 more threads
2019-03-20 09:27:35,382 : MainThread : INFO : worker thread finished; awaiting finish of 16 more threads
2019-03-20 09:27:35,383 : MainThread : INFO : worker thread finished; awaiting finish of 15 more threads
2019-03-20 09:27:35,385 : MainThread : INFO : worker thread finished; awaiting finish of 14 more threads
2019-03-20 09:27:35,386 : MainThread : INFO : worker thread finished; awaiting finish of 13 more threads
2019-03-20 09:27:35,389 : MainThread : INFO : worker thread finished; awaiting finish of 12 more threads
2019-03-20 09:27:35,395 : MainThread : INFO : worker thread finished; awaiting finish of 11 more threads
2019-03-20 09:27:35,399 : MainThread : INFO : worker thread finished; awaiting finish of 10 more threads
2019-03-20 09:27:35,400 : MainThread : INFO : worker th

2019-03-20 09:27:47,009 : MainThread : INFO : worker thread finished; awaiting finish of 18 more threads
2019-03-20 09:27:47,012 : MainThread : INFO : worker thread finished; awaiting finish of 17 more threads
2019-03-20 09:27:47,016 : MainThread : INFO : worker thread finished; awaiting finish of 16 more threads
2019-03-20 09:27:47,019 : MainThread : INFO : worker thread finished; awaiting finish of 15 more threads
2019-03-20 09:27:47,020 : MainThread : INFO : worker thread finished; awaiting finish of 14 more threads
2019-03-20 09:27:47,021 : MainThread : INFO : worker thread finished; awaiting finish of 13 more threads
2019-03-20 09:27:47,022 : MainThread : INFO : worker thread finished; awaiting finish of 12 more threads
2019-03-20 09:27:47,023 : MainThread : INFO : worker thread finished; awaiting finish of 11 more threads
2019-03-20 09:27:47,028 : MainThread : INFO : worker thread finished; awaiting finish of 10 more threads
2019-03-20 09:27:47,031 : MainThread : INFO : worker th

2019-03-20 09:27:58,699 : MainThread : INFO : worker thread finished; awaiting finish of 18 more threads
2019-03-20 09:27:58,701 : MainThread : INFO : worker thread finished; awaiting finish of 17 more threads
2019-03-20 09:27:58,702 : MainThread : INFO : worker thread finished; awaiting finish of 16 more threads
2019-03-20 09:27:58,706 : MainThread : INFO : worker thread finished; awaiting finish of 15 more threads
2019-03-20 09:27:58,709 : MainThread : INFO : worker thread finished; awaiting finish of 14 more threads
2019-03-20 09:27:58,715 : MainThread : INFO : worker thread finished; awaiting finish of 13 more threads
2019-03-20 09:27:58,719 : MainThread : INFO : worker thread finished; awaiting finish of 12 more threads
2019-03-20 09:27:58,722 : MainThread : INFO : worker thread finished; awaiting finish of 11 more threads
2019-03-20 09:27:58,724 : MainThread : INFO : worker thread finished; awaiting finish of 10 more threads
2019-03-20 09:27:58,725 : MainThread : INFO : worker th

...

2019-03-20 09:50:18,722 : MainThread : INFO : EPOCH 246 - PROGRESS: at 13.72% examples, 1140030 words/s, in_qsize 62, out_qsize 1
2019-03-20 09:50:19,722 : MainThread : INFO : EPOCH 246 - PROGRESS: at 32.76% examples, 1211123 words/s, in_qsize 62, out_qsize 1
2019-03-20 09:50:20,729 : MainThread : INFO : EPOCH 246 - PROGRESS: at 51.21% examples, 1246997 words/s, in_qsize 63, out_qsize 0
2019-03-20 09:50:21,737 : MainThread : INFO : EPOCH 246 - PROGRESS: at 70.10% examples, 1269908 words/s, in_qsize 64, out_qsize 0
2019-03-20 09:50:22,745 : MainThread : INFO : EPOCH 246 - PROGRESS: at 88.99% examples, 1269292 words/s, in_qsize 62, out_qsize 1
2019-03-20 09:50:23,066 : MainThread : INFO : worker thread finished; awaiting finish of 31 more threads
2019-03-20 09:50:23,070 : MainThread : INFO : worker thread finished; awaiting finish of 30 more threads
2019-03-20 09:50:23,075 : MainThread : INFO : worker thread finished; awaiting finish of 29 more threads
2019-03-20 09:50:23,080 : MainThrea

2019-03-20 09:50:29,800 : MainThread : INFO : EPOCH 248 - PROGRESS: at 14.59% examples, 1176785 words/s, in_qsize 64, out_qsize 0
2019-03-20 09:50:30,804 : MainThread : INFO : EPOCH 248 - PROGRESS: at 32.72% examples, 1204420 words/s, in_qsize 64, out_qsize 0
2019-03-20 09:50:31,820 : MainThread : INFO : EPOCH 248 - PROGRESS: at 50.26% examples, 1218145 words/s, in_qsize 63, out_qsize 0
2019-03-20 09:50:32,825 : MainThread : INFO : EPOCH 248 - PROGRESS: at 67.71% examples, 1221726 words/s, in_qsize 62, out_qsize 1
2019-03-20 09:50:33,825 : MainThread : INFO : EPOCH 248 - PROGRESS: at 86.92% examples, 1238239 words/s, in_qsize 63, out_qsize 0
2019-03-20 09:50:34,350 : MainThread : INFO : worker thread finished; awaiting finish of 31 more threads
2019-03-20 09:50:34,357 : MainThread : INFO : worker thread finished; awaiting finish of 30 more threads
2019-03-20 09:50:34,368 : MainThread : INFO : worker thread finished; awaiting finish of 29 more threads
2019-03-20 09:50:34,374 : MainThrea

2019-03-20 09:50:41,111 : MainThread : INFO : EPOCH 250 - PROGRESS: at 14.71% examples, 1199826 words/s, in_qsize 64, out_qsize 0
2019-03-20 09:50:42,113 : MainThread : INFO : EPOCH 250 - PROGRESS: at 32.41% examples, 1206753 words/s, in_qsize 63, out_qsize 0
2019-03-20 09:50:43,115 : MainThread : INFO : EPOCH 250 - PROGRESS: at 50.34% examples, 1232596 words/s, in_qsize 62, out_qsize 1
2019-03-20 09:50:44,140 : MainThread : INFO : EPOCH 250 - PROGRESS: at 69.78% examples, 1262422 words/s, in_qsize 64, out_qsize 0
2019-03-20 09:50:45,151 : MainThread : INFO : EPOCH 250 - PROGRESS: at 89.11% examples, 1268087 words/s, in_qsize 63, out_qsize 0
2019-03-20 09:50:45,514 : MainThread : INFO : worker thread finished; awaiting finish of 31 more threads
2019-03-20 09:50:45,516 : MainThread : INFO : worker thread finished; awaiting finish of 30 more threads
2019-03-20 09:50:45,524 : MainThread : INFO : worker thread finished; awaiting finish of 29 more threads
2019-03-20 09:50:45,531 : MainThrea

Saving model  models/doc2vec.model.2


FileNotFoundError: [Errno 2] No such file or directory: 'models/doc2vec.model.2'

# Using the doc2vec model

In [17]:
fullname = path.join(utils.Config.models_dir,utils.Config.nameModel+".2")
print("Loading model ", fullname)
modelDoc2Vec = doc2vec.Doc2Vec.load(fullname)
print("The model contains {0} vectors.".format(len(modelDoc2Vec.docvecs)))

2019-03-20 09:53:55,614 : MainThread : INFO : loading Doc2Vec object from models/doc2vec.model.2


Loading model  models/doc2vec.model.2


2019-03-20 09:53:55,942 : MainThread : INFO : loading vocabulary recursively from models/doc2vec.model.2.vocabulary.* with mmap=None
2019-03-20 09:53:55,945 : MainThread : INFO : loading wv recursively from models/doc2vec.model.2.wv.* with mmap=None
2019-03-20 09:53:55,946 : MainThread : INFO : loading docvecs recursively from models/doc2vec.model.2.docvecs.* with mmap=None
2019-03-20 09:53:55,947 : MainThread : INFO : loading trainables recursively from models/doc2vec.model.2.trainables.* with mmap=None
2019-03-20 09:53:55,948 : MainThread : INFO : loaded models/doc2vec.model.2


The model contains 1806 vectors.


In [18]:
"danna" in vocab_dict

False

In [19]:
# just a little test
w1 = "local-search"
words = modelDoc2Vec.wv.most_similar(w1,topn=50)
print("SIM {0:10s} = {1}".format(w1,[w[0] for w in words]))
print("-"*80)

w1 = "local-branching"
words = modelDoc2Vec.wv.most_similar(w1,topn=50)
print("SIM {0:10s} = {1}".format(w1,[w[0] for w in words]))
print("-"*80)

w1 = "relaxation-induced-neighborhood-search"
words = modelDoc2Vec.wv.most_similar(w1,topn=50)
print("SIM {0:10s} = {1}".format(w1,[w[0] for w in words]))
print("-"*80)

print("SIMIL = ", modelDoc2Vec.n_similarity(["variable-neighborhood-search"],["relaxation-induced-neighborhood-search"]))
#print(modelDoc2Vec.docvecs.doctags)
print(modelDoc2Vec.predict_output_word(context_words_list=corpus[5][0], topn=20)) #probability distribution of the center word given context words.
 #     .distance("hybrid", "matheuristic"))

2019-03-20 09:54:14,101 : MainThread : INFO : precomputing L2-norms of word weight vectors


SIM local-search = ['neighborhood', 'tabu-search', 'iterated-local-search', 'search', 'metaheuristic', 'neighbourhood', 'heuristic', 'displacement', 'swap', 'citizen', 'sonet', 'management', 'mmb', 'move', 'vrp', 'local-optimum', 'solution', 'ochi', 'intensification', 'restarting', 'usui', 'narrative', 'deconstruction', 'sampling', 'commissioning', 'tsukuba', 'variable-neighborhood-search', 'path-relinking', 'iteratively', 'viral', 'outlying', 'sea', 'rounding', 'adhikari', 'khe', 'guillaume', 'tar', 'anker', 'hearst', 'decoder', 'contracting', 'mitigating', 'simulated-annealing', 'barman', 'pitcher', 'instituto', 'erd', 'flip', 'wbl', 'acs']
--------------------------------------------------------------------------------
SIM local-branching = ['peripheral', 'variable-neighborhood-decomposition-search', 'rdj', 'achterberg', 'metaheuristic', 'soft-fixing', 'unblocking', 'timetable', 'ineffective', 'yaf', 'assmann', 'quoc', 'burnaby', 'ioi', 'evt', 'hso', 'hasta', 'borne', 'wtd', 'incumb

/home/mcaserta/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).


AttributeError: 'Doc2Vec' object has no attribute 'predict_output_word'

In [20]:
result = modelDoc2Vec.most_similar(positive=['bender', 'dantzig-wolfe', 'benders', 'lagrangean', 'relaxation'])#
#result = modelDoc2Vec.most_similar(positive=['local-branching', 'corridor-method', 'kernel-search', 'popmusic'],topn=5)
#result = modelDoc2Vec.most_similar(positive=['lagrangean', 'dantzig-wolfe'], negative=['matheuristic'])
print("result = ", result)

result =  [('column-generation', 0.33908915519714355), ('plus', 0.3340398371219635), ('dual', 0.33160948753356934), ('dedicating', 0.30587488412857056), ('master', 0.28753116726875305), ('ricci', 0.2835322618484497), ('cus', 0.2817022204399109), ('signal', 0.27730047702789307), ('large-scale', 0.27701249718666077), ('kanban', 0.2770102620124817)]


/home/mcaserta/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [21]:
modelDoc2Vec.wmdistance(corpus[0][0], corpus[1][0])

/home/mcaserta/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).
  """Entry point for launching an IPython kernel.
2019-03-20 09:55:18,037 : MainThread : INFO : Removed 15 and 17 OOV words from document 1 and 2 (respectively).
2019-03-20 09:55:18,039 : MainThread : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-03-20 09:55:18,042 : MainThread : INFO : built Dictionary(989 unique tokens: ['introduce', 'composed', 'kth', 'glover', 'application']...) from 2 documents (total 4108 corpus positions)


13.410735952207778

In [ ]:
v1=modelDoc2Vec.docvecs[corpus[0][1]]
v2=modelDoc2Vec.docvecs[corpus[1][1]]
print(corpus[1][1])
# use 1-similarity to get a distance
modelDoc2Vec.docvecs.similarity(corpus[0][1][0], corpus[1][1][0]) # note: get the tag, not the list
#modelDoc2Vec.docvecs.similarity(str(0), str(1))

# Populate Lists

We want to create two types of lists:
* List of bi-grams, three-grams, etc. Examples of bi-grams are population-based, corridor-method, etc.
* List of acronyms, e.g., TS, CM ,GA, RINS, etc.

In both cases, we will transform the text into the format "tabu-search", corridor-method". This implies that we need to check any two consecutive words (to indentify cases in which "tabu search" is used) as well as capital-letter acronyms (to identify cases in which "TS" is used.)

In [ ]:
def populate_ngrams(chunk, fNgrams, fAcrons, ngrams_dict, ngrams_new, acrons_dict, acrons_new):
    chunk = re.sub('[“”;\',.’´`\(\{\[\/\]\}\)=+]','', chunk)
    words = [w for w in word_tokenize(chunk) if w not in stop_words]
    for w in words:
        if w.isupper() and w.isalpha() and len(w) > 1 and w not in acrons_dict and w not in acrons_new:
            acrons_new.add(w)
            fAcrons.write(w+"\n")
    #words = [w.lower() for w in word_tokenize(chunk) if w not in stop_words]
    words = [w.lower() for w in words]
    i = 0
    corrected = []
    while i < len(words)-1: # skip last element (we could lose a few words here)
        w = words[i]
        if len(w) <= 1:
            i += 1
            continue
        if w[-1] == "-" and w[:-1].isalpha() and words[i+1].isalpha() and len(w[:-1]) >= 1:
            w = w[:-1] + words[i+1]
            i += 1
            print("** Concatenated word = ", w)
        # if we find a word that contains a hyphen, it must be a composite word, e.g. "worst-case"
        if len(re.findall("-",w)) > 0 and w[0] != "-" and w[-1] != "-" and w not in ngrams_dict and w not in ngrams_new:
            print("Adding '{0}' to list of n-grams".format(w))
            fNgrams.write(w+"\n") # add it to our list of n-grams
            ngrams_new.append(w)
            
        if (len(re.findall("-",w)) > 0 or w.isalpha()) and len(w) > 2: # eliminate short words
            corrected.append(w)
        i += 1
        
    return corrected
       


In [ ]:
ngrams_dict = create_dict(ngramsFile)
acrons_dict = create_dict(acronsFile)
ngrams_new  = []
acrons_new = {""} #"CM", "TS", "GA", "ACO", "POPMUSIC", "VNS", "VNDS", "RINS"} # a set


fNgrams = open(ngramsFile, "a") # use this in the first run, to populate the file
fAcrons = open(acronsFile, "a") # list of acronyms

#pdfFile = 'biblio/summary.pdf'
pdfFile = 'biblio/puchinger-05.pdf'

document = open(pdfFile, 'rb')
#Create resource manager
rsrcmgr = PDFResourceManager()
# Set parameters for analysis.
laparams = LAParams()
# Create a PDF page aggregator object.
device = PDFPageAggregator(rsrcmgr, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)

doc = []
biblio = False
for page in PDFPage.get_pages(document):
    interpreter.process_page(page)
    # receive the LTPage object for the page.
    layout = device.get_result()
    for element in layout:
        if isinstance(element, LTTextBoxHorizontal):
            el = element.get_text()
            words = populate_ngrams(el, fNgrams, fAcrons, ngrams_dict, ngrams_new, acrons_dict, acrons_new)
            
            # next, attempt to skip bibliography
            if len(words) == 0 and "Bibliography" in el:
                print("EL = ", el)
                print("Found Biblio? STOP Here" )
                biblio = True
                break
                
            for w in words:
                doc.append(w)
    if biblio:
        break
    #if i > 1:
    #    break
        
fNgrams.close()
fAcrons.close()

In [ ]:
acrons

# Testing

In [76]:
# download the file questions-words.txt to be used for comparing word embeddings
!wget https://raw.githubusercontent.com/tmikolov/word2vec/master/questions-words.txt


--2019-03-18 08:40:44--  https://raw.githubusercontent.com/tmikolov/word2vec/master/questions-words.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.132.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.132.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 603955 (590K) [text/plain]
Saving to: ‘questions-words.txt’

questions-words.txt 100%[===================>] 589.80K  --.-KB/s    in 0.1s    

2019-03-18 08:40:44 (4.32 MB/s) - ‘questions-words.txt’ saved [603955/603955]



In [89]:
import logging
from gensim.models import Word2Vec, KeyedVectors
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


def print_accuracy(model, questions_file):
    print('Evaluating...\n')
    acc = model.accuracy(questions_file)
    print("ACC =", acc)

    sem_correct = sum((len(acc[i]['correct']) for i in range(5)))
    sem_total = sum((len(acc[i]['correct']) + len(acc[i]['incorrect'])) for i in range(5))
    print("Sem total ", sem_total)
    sem_acc = 100*float(sem_correct)/sem_total
    print('\nSemantic: {:d}/{:d}, Accuracy: {:.2f}%'.format(sem_correct, sem_total, sem_acc))
    
    syn_correct = sum((len(acc[i]['correct']) for i in range(5, len(acc)-1)))
    syn_total = sum((len(acc[i]['correct']) + len(acc[i]['incorrect'])) for i in range(5,len(acc)-1))
    syn_acc = 100*float(syn_correct)/syn_total
    print('Syntactic: {:d}/{:d}, Accuracy: {:.2f}%\n'.format(syn_correct, syn_total, syn_acc))
    return (sem_acc, syn_acc)

word_analogies_file = 'questions-words.txt'
accuracies = []
print('\nLoading Gensim embeddings')

fullname = path.join(utils.Config.models_dir,utils.Config.nameModel)
print("Loading model ", fullname)
modelDoc2Vec = doc2vec.Doc2Vec.load(fullname)
print("The model contains {0} vectors.".format(len(modelDoc2Vec.docvecs)))
print('Accuracy for Word2Vec:')
accuracies.append(print_accuracy(modelDoc2Vec, word_analogies_file))


2019-03-18 08:48:58,486 : INFO : loading Doc2Vec object from models/doc2vec.model
2019-03-18 08:48:58,552 : INFO : loading wv recursively from models/doc2vec.model.wv.* with mmap=None
2019-03-18 08:48:58,554 : INFO : setting ignored attribute syn0norm to None
2019-03-18 08:48:58,555 : INFO : loading docvecs recursively from models/doc2vec.model.docvecs.* with mmap=None
2019-03-18 08:48:58,557 : INFO : setting ignored attribute cum_table to None
2019-03-18 08:48:58,558 : INFO : loaded models/doc2vec.model
2019-03-18 08:48:58,622 : INFO : precomputing L2-norms of word weight vectors



Loading Gensim embeddings
Loading model  models/doc2vec.model
The model contains 6 vectors.
Accuracy for Word2Vec:
Evaluating...



2019-03-18 08:48:58,739 : INFO : gram1-adjective-to-adverb: 0.0% (0/56)
2019-03-18 08:48:58,763 : INFO : gram2-opposite: 0.0% (0/2)
2019-03-18 08:48:58,962 : INFO : gram3-comparative: 0.0% (0/182)
2019-03-18 08:48:59,066 : INFO : gram4-superlative: 0.0% (0/56)
2019-03-18 08:48:59,132 : INFO : gram5-present-participle: 3.3% (1/30)
2019-03-18 08:48:59,149 : INFO : gram6-nationality-adjective: 0.0% (0/2)
2019-03-18 08:48:59,208 : INFO : gram7-past-tense: 0.0% (0/30)
2019-03-18 08:48:59,227 : INFO : gram8-plural: 0.0% (0/2)
2019-03-18 08:48:59,316 : INFO : gram9-plural-verbs: 0.0% (0/42)
2019-03-18 08:48:59,317 : INFO : total: 0.2% (1/402)


ACC = [{'section': 'capital-common-countries', 'incorrect': [], 'correct': []}, {'section': 'capital-world', 'incorrect': [], 'correct': []}, {'section': 'currency', 'incorrect': [], 'correct': []}, {'section': 'city-in-state', 'incorrect': [], 'correct': []}, {'section': 'family', 'incorrect': [], 'correct': []}, {'section': 'gram1-adjective-to-adverb', 'incorrect': [('COMPLETE', 'COMPLETELY', 'EFFICIENT', 'EFFICIENTLY'), ('COMPLETE', 'COMPLETELY', 'FREE', 'FREELY'), ('COMPLETE', 'COMPLETELY', 'OBVIOUS', 'OBVIOUSLY'), ('COMPLETE', 'COMPLETELY', 'POSSIBLE', 'POSSIBLY'), ('COMPLETE', 'COMPLETELY', 'PRECISE', 'PRECISELY'), ('COMPLETE', 'COMPLETELY', 'TYPICAL', 'TYPICALLY'), ('COMPLETE', 'COMPLETELY', 'USUAL', 'USUALLY'), ('EFFICIENT', 'EFFICIENTLY', 'FREE', 'FREELY'), ('EFFICIENT', 'EFFICIENTLY', 'OBVIOUS', 'OBVIOUSLY'), ('EFFICIENT', 'EFFICIENTLY', 'POSSIBLE', 'POSSIBLY'), ('EFFICIENT', 'EFFICIENTLY', 'PRECISE', 'PRECISELY'), ('EFFICIENT', 'EFFICIENTLY', 'TYPICAL', 'TYPICALLY'), ('EFFIC

ZeroDivisionError: float division by zero